In [1]:
import pandas as pd
import awswrangler as awr
import openpyxl


In [2]:


path_to_query = r"C:\Users\raphael.almeida\Documents\Projetos\Seguro de Vida Motorista\listagem_mestra_ajustada.sql"

with open (path_to_query,'r') as reading:
    query_readed = reading.read()

df = awr.athena.read_sql_query(query_readed,'silver')

In [23]:
df.info()
len(df)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 130930 entries, 0 to 130929
Data columns (total 25 columns):
 #   Column           Non-Null Count   Dtype 
---  ------           --------------   ----- 
 0   cooperativa      130930 non-null  string
 1   unidade          130926 non-null  string
 2   cliente          130930 non-null  string
 3   matricula        130930 non-null  Int64 
 4   conjunto         130930 non-null  Int64 
 5   coverage         130930 non-null  Int64 
 6   beneficio        130930 non-null  string
 7   motorista        13578 non-null   string
 8   id_placa         88154 non-null   Int64 
 9   placa            107910 non-null  string
 10  chassi           88154 non-null   string
 11  renavam          88141 non-null   string
 12  ativacao         130924 non-null  object
 13  inicio_vig       130924 non-null  object
 14  fim_vig          130924 non-null  object
 15  status           130930 non-null  string
 16  status_coverage  130930 non-null  string
 17  benefits_i

130930

placas totais

In [21]:
df['placa'].drop_duplicates().count()

31621

seguro de vida geral

In [19]:
df_seguro_de_vida = df[df['benefits_id'].isin([6,28,44])]
len(df_seguro_de_vida)

#print(len(df_seguro_de_vida))
#df_seguro_de_vida.head()
#print(len(df_seguro_de_vida.drop_duplicates(subset='placa')))
#df_seguro_de_vida['placa'].drop_duplicates().count()

12144

seguro de vida de motorista

In [25]:
df_seguro_de_vida_motorista = df_seguro_de_vida[df_seguro_de_vida['faixa_valor'].str.contains('MOTORISTA', na=False)]
len(df_seguro_de_vida_motorista)

9625

In [27]:
df_cascos_veiculo = df[df['benefits_id'].isin([2, 24, 40])]
len(df_cascos_veiculo)

17148

base_analise

In [4]:

#excluir coluna de e-mail pois está diferenciando na análise
#df = df.drop('e_mail', axis=1)
df = df.drop_duplicates()

#pegar apenas os conjuntos obrigatorios de seguro de vida
beneficios_conjuntos = df.groupby(['conjunto', 'cooperativa'])['benefits_id'].unique()
conjuntos_validos = beneficios_conjuntos[(beneficios_conjuntos.apply(lambda x: {2,6}.issubset((x))))|
                                         (beneficios_conjuntos.apply(lambda x: {24,28}.issubset((x))))|
                                         (beneficios_conjuntos.apply(lambda x: {40,44}.issubset((x))))].index

df_obrigatorio = df[df.set_index(['conjunto','cooperativa']).index.isin(conjuntos_validos)].copy()

#criar colunas para posterior agregação
df_obrigatorio['casco'] = df_obrigatorio['benefits_id'].apply(lambda x: x if (x == 2 or x == 24 or x == 40) else pd.NA)
#df_obrigatorio['seg_vida'] = df_obrigatorio['benefits_id'].apply(lambda x: x if (x==6 or x==28 or x==44) else pd.NA)


df_obrigatorio['seg_vida_motorista'] = df_obrigatorio.apply(lambda row: 
                                                  row['benefits_id'] if (row['benefits_id'] in [6,28,44] and 'MOTORISTA' in row['faixa_valor'])
else pd.NA, axis=1)

df_obrigatorio['seg_vida_geral'] = df_obrigatorio.apply(lambda row: 
                                                  row['benefits_id'] if (row['benefits_id'] in [6,28,44] and 'MOTORISTA' not in row['faixa_valor'])
else pd.NA, axis=1)

#filtrando apenas os que fazem sentido para a análise (casco e seguro de vida)
df_obrigatorio_filtrado = df_obrigatorio[df_obrigatorio['benefits_id'].isin([2,6,24,28,40,44])]

In [5]:
df_obrigatorio_filtrado.head()

,cooperativa,unidade,cliente,matricula,conjunto,coverage,beneficio,motorista,id_placa,placa,...,categoria,tipo_categoria,faixa_valor,tabela_ativa,tipo_veiculo,tipo,e_mail,casco,seg_vida_motorista,seg_vida_geral
0,Stcoop,UNIDADE SINOP,FABIO TURRA JAEGER,21573,13608,84648,CASCO (VEÍCULO),<NA>,17310,RLJ1C15,...,CAVALO,LC: 100% - CP: 4% - SCANIA - 2024 ATÉ 2021,"De R$600.000,01 até R$625.000,00",S,Cavalo,RenovaÃ§Ã£o,<NA>,24,<NA>,<NA>
1,Stcoop,UNIDADE CASCAVEL,RICARDO ROSA AMORIM EIRELI,21578,13610,84655,CASCO (VEÍCULO),<NA>,19062,QAY6H04,...,CAVALO,LC: 100% - CP: 4% - SCANIA - 2024 ATÉ 2021,"De R$625.000,01 até R$650.000,00",S,Cavalo,RenovaÃ§Ã£o,<NA>,24,<NA>,<NA>
5,Stcoop,UNIDADE RONDONÓPOLIS,VILMAR ALEXANDRE DA SILVA TRANSPORTES,11931,13612,84672,SEGURO DE VIDA,VILMAR ALEXANDRE DA SILVA,<NA>,<NA>,...,PADRÃO,PADRÃO,MOTORISTA - Seguro de Vida via Rep. ou Rep. do...,N,<NA>,RenovaÃ§Ã£o,<NA>,<NA>,28,<NA>
8,Stcoop,UNIDADE CURITIBA,A. SAID TRANSPORTES LTDA - ME,6400,13616,84691,CASCO (VEÍCULO),<NA>,18914,BDV1A44,...,CAVALO,LC: 100% - CP: 4% - DAF E MAN - 2024 ATÉ 2019,"De R$350.000,01 até R$375.000,00",S,Cavalo,RenovaÃ§Ã£o,<NA>,24,<NA>,<NA>
10,Stcoop,UNIDADE ITAJAI,SANDRO LUIZ VIEIRA,2312,13618,84709,SEGURO DE VIDA,SANDRO LUIZ VIEIRA,<NA>,<NA>,...,PADRÃO,PADRÃO,MOTORISTA - Seguro de Vida via Rep. ou Rep. do...,N,<NA>,RenovaÃ§Ã£o,<NA>,<NA>,28,<NA>


In [7]:
df_obrigatorio_filtrado['seg_vida_motorista'].count()

9611

conjuntos_analise

In [14]:


df_analise = df_obrigatorio_filtrado.groupby(['cooperativa','unidade','matricula','conjunto','e_mail']).agg(
    qtd_seg_vida_motorista = ('seg_vida_motorista', 'count'),
    qtd_casco = ('casco', 'count'),
    qtd_seg_vida_geral = ('seg_vida_geral', 'count')
).reset_index()

#df_analise.drop_duplicates('e_mail', keep='first')
df_analise.sort_values(by='conjunto', ascending=False)

# df_analysis=df_analise[(df_analise['qtd_motorista']!=df_analise['qtd_casco'])|
#                       (df_analise['qtd_motorista']!=df_analise['qtd_seg_vida'])|
#                       (df_analise['qtd_casco']!=df_analise['qtd_seg_vida'])]

df_analysis = df_analise[df_analise['qtd_seg_vida_motorista']<df_analise['qtd_casco']]

#df_analysis = df_analise[row, ['cooperativa','unidade','matricula','conjunto','qtd_motoristas','qtd_cascos','qtd_segvida']]

# file_path  = r'C:\Users\raphael.almeida\Documents\Projetos\Seguro de Vida Motorista\conjuntos_analise.xlsx'
# df_analysis.to_excel(file_path, engine = 'openpyxl', index = False)

# file_path  = r'C:\Users\raphael.almeida\Documents\Projetos\Seguro de Vida Motorista\conjuntos_variados.xlsx'
# df_analise.to_excel(file_path, engine = 'openpyxl', index = False)



parametros de e-mail

In [15]:
#-----------------LOAD
#Function to send e-mails

#p_email,

def enviar_email(p_unidade, p_empresa, tabela_html):    
    outlook = win32.Dispatch('outlook.application')  #integrating python with outlook 

    #creating e-mail variable to use
    email = outlook.CreateItem(0)

    #configuring the e-mail - ALTEREI PARA TESTE
    email.To = "dados13@grupounus.com.br; suelen.martucci@transdesk.com.br"
    email.Subject = "[PENDÊNCIA DE SEGURO DE VIDA - MOTORISTAS] - Análise de Dados"
    email.HTMLBody = f"""

    <p>Prezado(a),</p>

    <p>A {p_unidade} ({p_empresa}) possui pendências de cadastramento de seguro de vida <b>para motoristas</b>.</p>

    <p>A quantidade de coberturas concernentes ao casco é maior do que a quantidade de coberturas de seguro de vida <b>para motoristas</b>.</p>

    <p>{tabela_html}</p>

    <p>Favor, cadastrar os seguros de vida <b>para motoristas</b> remanescentes para cômputo na base de dados da empresa, obrigado!</p>

    <p>Atenciosamente,</p>

    <p>Equipe Análise de Dados - Grupo Unus</p>

    <p><i>Este é um e-mail automático, por favor não responda</i></p>

    """

    email.Send()

    print("Email enviado")

função envio email real

In [ ]:
df_analysis3 = df_analysis2.drop('qtd_motorista', axis=1)
df_email = df_analysis3[['cooperativa','unidade','e_mail']].drop_duplicates('e_mail', keep='first')

for _, row in df_email.iterrows():
    iEmail = row['e_mail']
    iUnidade = row['unidade']
    iCooperativa = row['cooperativa']

    df_filtrado = df_analysis3[(df_analysis3['unidade']==iUnidade) & (df_analysis3['cooperativa']==iCooperativa)]

    if df_filtrado.empty:
        print(f"Nenhum dado encontrado para {iUnidade} - {iCooperativa}. E-mail não enviado.")
        continue  

    df_tabela = df_filtrado.to_html(index=False, border=1)

    # if __name__ == '__main__':
    #     enviar_email(iEmail, iUnidade, iCooperativa, df_tabela)

    print(f"E-mail enviado para {iEmail}: {iUnidade}({iCooperativa})")



função envio e-mail teste

In [16]:

# df_analysis['e_mail']='dados13@grupounus.com.br'
df_analysis.drop(columns='qtd_seg_vida_geral', axis=1, inplace=True)
df_email = df_analysis[['cooperativa','unidade','e_mail']].drop_duplicates('unidade', keep='first')
df_email

for _, row in df_email.iterrows():
    iEmail = row['e_mail']
    iUnidade = row['unidade']
    iCooperativa = row['cooperativa']

    df_filtrado = df_analysis[(df_analysis['unidade']==iUnidade) & (df_analysis['cooperativa']==iCooperativa)]

    if df_filtrado.empty:
        print(f"Nenhum dado encontrado para {iUnidade} - {iCooperativa}. E-mail não enviado.")
        continue  

    df_tabela = df_filtrado.to_html(index=False, border=1, classes = "tabela")

    df_tabela = f"""
    <style>
        .tabela {{
            width: 100%;
            border-collapse: collapse;
        }}
        .tabela th, .tabela td {{
            text-align: center;  
            padding: 8px;
            border: 1px solid black;
        }}
    </style>
    {df_tabela}
    """
#iEmail, 
    if __name__ == '__main__':
        enviar_email(iUnidade, iCooperativa, df_tabela)

    print(f"E-mail enviado para {iEmail}: {iUnidade}({iCooperativa})")


C:\Users\raphael.almeida\AppData\Local\Temp\ipykernel_31672\616413508.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_analysis.drop(columns='qtd_seg_vida_geral', axis=1, inplace=True)


Email enviado
E-mail enviado para contasapagar.unus@gmail.com: TS CONSULTORIA EM TRANSPORTES LTDA(Stcoop)
Email enviado
E-mail enviado para unidade.cascavel@transdesk.com.br: UNIDADE CASCAVEL(Stcoop)
Email enviado
E-mail enviado para unidade.curitiba@transdesk.com.br: UNIDADE CURITIBA(Stcoop)
Email enviado
E-mail enviado para unidade.sinop@transdesk.com.br: UNIDADE SINOP(Stcoop)
Email enviado
E-mail enviado para unidade.vilhena@transdesk.com.br: UNIDADE VILHENA(Stcoop)
Email enviado
E-mail enviado para unidade.campogrande@transdesk.com.br: UNIDADE CAMPO GRANDE(Viavante)
Email enviado
E-mail enviado para unidade.carazinho@transdesk.com.br: UNIDADE CARAZINHO(Viavante)
Email enviado
E-mail enviado para unidade.cuiaba@transdesk.com.br: UNIDADE CUIABA(Viavante)
Email enviado
E-mail enviado para unidade.curitiba-vendaonline@transdesk.com.br: UNIDADE CURITIBA - VENDA ONLINE(Viavante)
Email enviado
E-mail enviado para unidade.dourados@transdesk.com.br: UNIDADE DOURADOS(Viavante)
Email enviado
